In [66]:
# Make sure libraries are reloaded when this cell is run
%load_ext autoreload
%autoreload 2

from astropy import units as u

from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import OrbitPlotter3D
import copy
import CustomManeuvres

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
# Define the two coplanar debris
a = 3000+6371 << u.km
ecc = 0 << u.one
inc = 0 << u.deg
raan = 0 << u.deg
argp = 0 << u.deg
nu = 11 << u.deg

deb_1 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

a = 5000+6371 << u.km
nu = 40 << u.deg

deb_2 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

# Init the otv at debris 1
otv = copy.copy(deb_1)

In [39]:
print(deb_1.epoch.iso)
print(deb_2.epoch.iso)
print(otv.epoch.iso)

2000-01-01 12:00:00.000
2000-01-01 12:00:00.000
2000-01-01 12:00:00.000


In [68]:
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

In [69]:
# Calculate a basic homnann
import CustomManeuvres
hoh_phas = CustomManeuvres.hohmann_with_phasing(otv, deb_2)
hoh_phas

mean:  2.7364204967860117 1 / s
angle 0.4051721568037814 rad
Target Delta:  23.214654561069477 deg
current_delta :  29.0 deg
t_1:  576.0387077994128 s
orbit_i.nu:  33.970173784843595 deg
new delta:  23.214654561069477 deg


Number of impulses: 2, Total cost: 0.599873 km / s

In [70]:
hoh_phas.impulses

((<Quantity 576.0387078 s>,
  <Quantity [-171.64931612,  254.76655237,    0.        ] m / s>),
 (<Quantity 5255.47453664 s>,
  <Quantity [ 163.53637938, -242.72511248,    0.        ] m / s>))

In [71]:
# Get the transfer time of the hoh_phas
transfer_time = hoh_phas.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)

# Propagate the otv using the calculated thrust sequence
transfer_orbit, otv = otv.apply_maneuver(hoh_phas, intermediate=True)

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)
frame.plot(transfer_orbit)



In [65]:
print("OTV ANGLE: ", otv.nu.to(u.deg))
print("DEB2 ANGLE: ", deb_2.nu.to(u.deg))
print('difference: ', (otv.nu - deb_2.nu).to(u.deg))
print('OTV.a: ', otv.a)
print('Deb2.a: ', deb_2.a)

OTV ANGLE:  -146.02982621515653 deg
DEB2 ANGLE:  -146.0298262151564 deg
difference:  -1.526666247102488e-13 deg
OTV.a:  11371.000000000004 km
Deb2.a:  11371.0 km


178.73